In [1]:
import sys
import numpy as np
import pandas as pd
from multiprocessing import Process
sys.path.insert(0, '../../')

In [2]:
from library.notebook_api.data_loader import CombinedDataLoader, ModelDataLoader

In [3]:
from library.source_data.feature_extractor import AudioFeatureExtractor
from library.source_data.parallel_processor import AudioParallelProcessor


## Balanced Extraction
This scenario runs extraction on a subset of the large version of the data set, filtering to rows matching a list of provided genres, and also trying to balance the dataset through obtaining an equal sized sample from each genre 

In [6]:
#instantate data_loader and the dataframes it makes available 
in_scope_labels = ['rock', 'electronic', 'hiphop', 'classical', 'jazz','country']
data_loader = CombinedDataLoader('large', in_scope_labels)
#df_balanced = data_loader.get_data_sampled_by_label(500)
df_balanced = data_loader.get_data_sampled_by_label(5000)

len(df_balanced)

tracks in meta 29701
tracks with files available in project_data_path:  29701
tracks with top level genres available 29701
tracks with genres and files (df_filtered) 29701


15947

to update using incremental batches, the code below checks the current context of the Model data version and determines the list of tracks that have yet to been process from the source

In [11]:
#get dataframe of current existing track ids from ModelDataLoader
current_data = ModelDataLoader('006')
current_data_df = current_data.df
current_track_list = current_data_df[['track_id','file_available']].copy()
#rename file_available column to differentiate it 
current_track_list.rename(columns={'file_available':'file_available_in_input'}, inplace=True)
#join this to medium source data to figure out tracks we don't have 
df_filtered_with_current_track_list = pd.merge(df_balanced,current_track_list,on='track_id',how='left')
#boolean indexer for the incremental files
incremental = (df_filtered_with_current_track_list['file_available_in_input'].isnull()) &(df_filtered_with_current_track_list['file_available'] ==1)

df_filtered_incremental = df_filtered_with_current_track_list[incremental].copy()

print(len(df_filtered_incremental), 'files left to process ')

9504 files left to process 


In [13]:
SAMPLE_RATE  = 22500
SECONDS = 25
#these batches include mfccs and log_melspectrogram, downsampled to 22500 , truncated to 25 seconds 
#batch_mfcc = AudioParallelProcessor(df_filtered_incremental.iloc[0:1200],version = '006', batch=2, threads = 4,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)
#batch_mfcc2 = AudioParallelProcessor(df_filtered_incremental.iloc[1201:],version = '006', batch=3, threads = 4,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)
#batch_mfcc = AudioParallelProcessor(df_filtered_incremental.iloc[0:2000],version = '006', batch=4, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)
#batch_mfcc2 = AudioParallelProcessor(df_filtered_incremental.iloc[2001:4000],version = '006', batch=5, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)

batch_mfcc = AudioParallelProcessor(df_filtered_incremental.iloc[0:2000],version = '006', batch=6, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)
batch_mfcc2 = AudioParallelProcessor(df_filtered_incremental.iloc[2001:4000],version = '006', batch=7, threads = 5,sample_rate=SAMPLE_RATE,start_sample=0,end_sample=SAMPLE_RATE*SECONDS)



In [14]:
batch_mfcc.execute()

populate thread process for thread 1  0 400 400
populate thread process for thread 2  401 800 399
populate thread process for thread 3  801 1200 399
populate thread process for thread 4  1201 1600 399
populate thread process for thread 5  1601 2000 399
start process thread 1
start process thread 2


python(1727) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


start process thread 3


python(1728) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


start process thread 4


...

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/mu

In [15]:
batch_mfcc2.execute()

populate thread process for thread 1  0 399 399
populate thread process for thread 2  400 798 398
populate thread process for thread 3  799 1197 398
populate thread process for thread 4  1198 1596 398
populate thread process for thread 5  1597 1999 402
start process thread 1
start process thread 2


python(2434) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


start process thread 3


python(2435) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


start process thread 4


...

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/mu

In [16]:
data_check = ModelDataLoader('006')

In [17]:
len(data_check.df)

11124

In [18]:
mfcc = data_check.get_mfcc()
log_melspectrogram = data_check.get_log_melspectrogram()

In [19]:
mfcc.shape, log_melspectrogram.shape

((11124,), (11124,))

In [20]:
mfcc[0].shape, log_melspectrogram[0].shape

((13, 1099), (128, 1099))

In [21]:
data_check.class_distribution

,label,proportion
0,rock,48.624595
1,electronic,33.692916
2,hiphop,5.267889
3,classical,5.007192
4,jazz,4.764473
5,country,2.642934


## Feature Extraction fma_large

### Load Metadata
pass the optional fma_audio size argument to Combined data loader to reference the fma_large file directory as part of combinding fma data with gtzan data

In [5]:
#instantate data_loader and the dataframes it makes available 
data_loader = CombinedDataLoader('large')
df = data_loader.df
df_files_available = data_loader.df_files_available
df_genres_available = data_loader.df_genres_available
df_filtered = data_loader.df_filtered

#get dataframe of current existing track ids from ModelDataLoader
current_data = ModelDataLoader('005')
current_data_df = current_data.df
current_track_list = current_data_df[['track_id','file_available']].copy()
#rename file_available column to differentiate it 
current_track_list.rename(columns={'file_available':'file_available_in_input'}, inplace=True)
#join this to medium source data to figure out tracks we don't have 
df_filtered_with_current_track_list = pd.merge(df_filtered,current_track_list,on='track_id',how='left')
#boolean indexer for the incremental files
incremental = (df_filtered_with_current_track_list['file_available_in_input'].isnull()) &(df_filtered_with_current_track_list['file_available'] ==1)

df_filtered_incremental = df_filtered_with_current_track_list[incremental].copy()

print(len(df_filtered_incremental), 'files left to process ')

tracks in meta 107574
tracks with files available in project_data_path:  107574
tracks with top level genres available 50598
tracks with genres and files (df_filtered) 50598
25720 files left to process 


In [8]:
in_scope_labels = ['rock', 'electronic', 'hiphop', 'classical', 'jazz','country']
in_scope_labels = [ 'hiphop', 'classical', 'jazz','country']

df_incremental_under_represented = df_filtered_incremental[df_filtered_incremental['label'].apply(lambda label: True if label in in_scope_labels else False)]

df_incremental_under_represented['label'].value_counts()

label
hiphop       2173
classical     444
jazz          255
country       202
Name: count, dtype: int64

### Extract and save numerical features in parquet and mfcc as npy nd array

In [10]:
batch_mfcc = AudioParallelProcessor(df_incremental_under_represented,version = '005', batch=12, threads = 4)

#batch_mfcc = AudioParallelProcessor(df_filtered_incremental.iloc[0:2000],version = '005', batch=11, threads = 4)
#batch_mfcc2 = AudioParallelProcessor(df_filtered_incremental.iloc[2001:4000],version = '005', batch=12, threads = 4)
#batch_mfcc3 = AudioParallelProcessor(df_filtered_incremental.iloc[4001:6000],version = '005', batch=13, threads = 4)

In [11]:
batch_mfcc.execute()

populate thread process for thread 1  0 768 768
populate thread process for thread 2  769 1536 767
populate thread process for thread 3  1537 2304 767
populate thread process for thread 4  2305 3074 769
start process thread 1
start process thread 2
start process thread 3
start process thread 4


[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:58: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(PROJECT_ABSOLUTE_PATH+file_name, sr=None)
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


failure in librosa.load


[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 33361.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1365] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 187493.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1365] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
Note: Illegal Audio-MPEG-Header

failure in librosa.load
failure in librosa.load
failure in librosa.load
failure in librosa.load


...

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:118: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:118: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:118: FutureWarning: librosa.beat.tempo
	This 

Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_save['fma_genres'] = to_save['fma_genres'].astype('string')
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_save['fma_genres_all'] = to_sa

In [20]:
batch_mfcc2.execute()

populate thread process for thread 1  0 499 499
populate thread process for thread 2  500 998 498
populate thread process for thread 3  999 1497 498
populate thread process for thread 4  1498 1999 501
start process thread 1


start process thread 2


python(71219) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


start process thread 3


python(71224) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [21]:
batch_mfcc3.execute()

populate thread process for thread 1  0 499 499
populate thread process for thread 2  500 998 498
populate thread process for thread 3  999 1497 498
populate thread process for thread 4  1498 1999 501
start process thread 1
start process thread 2


python(72500) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


start process thread 3


python(72501) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(72502) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


start process thread 4


...

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=944
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=967
  warnings.warn(


Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Error processing audio: when mode='interp', width=9 cannot exceed data.shape[axis]=1


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [22]:
m = ModelDataLoader('005')

In [23]:
m.df.head()
len(m.df)

24878

In [24]:
m.get_mfcc()[0:2]
#len(m.get_mfcc())
#[data.shape for data in m.get_mfcc()]

array([array([[-537.0103   , -395.64932  , -293.0882   , ..., -176.54858  ,
               -213.63383  , -238.7976   ],
              [   0.       ,  132.2462   ,  179.51065  , ...,  130.68646  ,
                155.1424   ,  160.9313   ],
              [   0.       ,   24.956886 ,   -2.3601766, ...,   -6.587591 ,
                 -0.6357365,   -1.7268903],
              ...,
              [   0.       ,   -9.952766 ,   -5.961585 , ...,   -4.0423937,
                 -2.8745453,   -5.442705 ],
              [   0.       ,  -11.037275 ,   -9.431261 , ...,    1.3861768,
                 -1.1699064,  -10.751028 ],
              [   0.       ,  -11.004683 ,   -8.728918 , ...,   -6.6627665,
                -11.070683 ,  -14.524673 ]], dtype=float32)                ,
       array([[-556.89716   , -317.3924    , -199.16246   , ..., -189.21332   ,
               -182.22647   , -165.0411    ],
              [   0.805253  ,  135.43683   ,  163.46066   , ...,  198.40375   ,
                188.01

### Add To Existing Numerical Featurres
In this case we already have features based on previous runs in model input version 3. Instead of writing out a new version,  we'll figure out the incremental files in that are in the source yet not yet in destination, and then write out more features for these files to the same version folder as new batches 

In [15]:
#get dataframe of current existing track ids from ModelDataLoader
current_v3 = ModelDataLoader('003')
curret_v3_df = current_v3.df
current_track_list = curret_v3_df[['track_id','file_available']].copy()
#rename file_available column to differentiate it 
current_track_list.rename(columns={'file_available':'file_available_in_input'}, inplace=True)
#join this to medium source data to figure out tracks we don't have 
df_filtered_with_current_track_list = pd.merge(df_filtered,current_track_list,on='track_id',how='left')
#boolean indexer for the incremental files
incremental = (df_filtered_with_current_track_list['file_available_in_input'].isnull()) &(df_filtered_with_current_track_list['file_available'] ==1)

df_filtered_incremental = df_filtered_with_current_track_list[incremental].copy()

print(len(df_filtered_incremental), 'files left to process ')


2387 files left to process 


In [16]:
#instantiate a few more parallel processes to add more features for incrental files to existing version 
batch2 = AudioParallelProcessor(df_filtered_incremental,version = '003', batch=17, threads = 6)
#batch2 = AudioParallelProcessor(df_filtered_incremental.iloc[0:3000],version = '003', batch=14, threads = 6)

#batch3 = AudioParallelProcessor(df_filtered_incremental.iloc[3001:6000],version = '003', batch=15, threads = 6)
#batch4 = AudioParallelProcessor(df_filtered_incremental.iloc[6001:9000],version = '003', batch=16, threads = 6)


In [17]:

batch2.execute()

populate thread process for thread 1  0 397 397
populate thread process for thread 2  398 794 396
populate thread process for thread 3  795 1191 396
populate thread process for thread 4  1192 1588 396
populate thread process for thread 5  1589 1985 396
populate thread process for thread 6  1986 2387 401
start process thread 1
start process thread 2
start process thread 3
start process thread 4
start process thread 5
start process thread 6


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:39: UserWarning: PySoundFile failed.

In [10]:
batch3.execute()

populate thread process for thread 1  0 499 499
populate thread process for thread 2  500 998 498
populate thread process for thread 3  999 1497 498
populate thread process for thread 4  1498 1996 498
populate thread process for thread 5  1997 2495 498
populate thread process for thread 6  2496 2999 503
start process thread 1
start process thread 2
start process thread 3
start process thread 4
start process thread 5
start process thread 6


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:129: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:129: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onse

In [11]:
batch4.execute()

populate thread process for thread 1  0 499 499
populate thread process for thread 2  500 998 498
populate thread process for thread 3  999 1497 498
populate thread process for thread 4  1498 1996 498
populate thread process for thread 5  1997 2495 498
populate thread process for thread 6  2496 2999 503
start process thread 1
start process thread 2
start process thread 3
start process thread 4
start process thread 5
start process thread 6


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:129: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:129: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:129: FutureWarning: librosa.beat.tempo
	This 

In [9]:
ModelDataLoader('003').df.count()

index                                 49759
track_id                              49759
dataset                               49759
audio_path                            49759
label                                 49759
fma_genre_top                         49759
fma_genres                            49759
fma_genres_all                        49759
file_available                        49759
sampling_rate                         49759
features                              49759
spectral_centroids_mean               49759
spectral_centroids_delta_mean         49759
spectral_centroids_accelerate_mean    49759
spectral_bandwidth_mean               49759
spectral_rolloff_mean                 49759
zero_crossing_rate_mean               49759
rms_mean                              49759
chroma_stft_mean                      49759
mfccs_mean                            49759
onset                                 49759
tempo                                 49759
contrast                        

In [10]:
ModelDataLoader('003').df.groupby('label')['label'].count().sort_values(ascending=False)

label
rock                  14291
experimental          10383
electronic             9148
hiphop                 3632
folk                   2783
pop                    2240
instrumental           2062
international          1473
classical              1316
jazz                    669
oldtime / historic      547
spoken                  413
country                 294
soulrnb                 274
blues                   210
easy listening           24
Name: label, dtype: int64

In [14]:
ModelDataLoader('003').class_distribution

,label,proportion
0,rock,28.713364
1,experimental,20.640518
2,electronic,18.257244
3,hiphop,7.485843
4,folk,5.523636
5,pop,4.552903
6,instrumental,4.189915
7,international,2.887308
8,classical,2.725519
9,jazz,1.373131


## Feature Extraction fma_mall

### Load Initial Meta

In [4]:
#instantate data_loader and the dataframes it makes available 
data_loader = CombinedDataLoader()
df = data_loader.df
df_files_available = data_loader.df_files_available
df_genres_available = data_loader.df_genres_available
df_filtered = data_loader.df_filtered

tracks in meta 107574
tracks with files available in project_data_path:  26000
tracks with top level genres available 50598
tracks with genres and files (df_filtered) 26000


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/notebook_api/data_loader.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  self.df_filtered = self.df_files_available[self.df.label.isnull() ==False ]


In [5]:
df_filtered.count()

track_id          26000
dataset           26000
audio_path        26000
label             26000
fma_genre_top     26000
fma_genres        26000
fma_genres_all    26000
file_available    26000
dtype: int64

### Running A Single Batch of parallel processes
Latest approach is to process via a parallel processor which runs multiple extraction tasks in parallel 
and puts outputs to separate parquet folders that are all loadable with data_loader. In this example we'll run just one batch on the full fma_small data.  It takes about an hour with 6 threads, but time varies on the CPU

In [6]:
#instantiate source data df to process
#batch_input = df_filtered.sample(100).copy() #subset of full data for demostration
batch_input = df_filtered.copy()#full data 

#instantate a parallel processing batch 
batch = AudioParallelProcessor(batch_input,version = '003', batch=1, threads = 6)

In [7]:
#sample with 100 and 6 threads 
#execute the parallel processes 
batch.execute()

populate thread process for thread 1  0 16 16
populate thread process for thread 2  17 32 15
populate thread process for thread 3  33 48 15
populate thread process for thread 4  49 64 15
populate thread process for thread 5  65 80 15
populate thread process for thread 6  81 100 19
start process thread 1
start process thread 2
start process thread 3
start process thread 4
start process thread 5
start process thread 6


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:122: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:122: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:122: FutureWarning: librosa.beat.tempo
	This 

### Load Data to validate 

In [9]:
full_model_data = ModelDataLoader('003')

In [10]:
full_model_data.df.count()

index                                 8991
track_id                              8991
dataset                               8991
audio_path                            8991
label                                 8991
fma_genre_top                         8991
fma_genres                            8991
fma_genres_all                        8991
file_available                        8991
sampling_rate                         8991
features                              8991
spectral_centroids_mean               8991
spectral_centroids_delta_mean         8991
spectral_centroids_accelerate_mean    8991
spectral_bandwidth_mean               8991
spectral_rolloff_mean                 8991
zero_crossing_rate_mean               8991
rms_mean                              8991
chroma_stft_mean                      8991
mfccs_mean                            8991
onset                                 8991
tempo                                 8991
contrast                              8991
tonnetz    

In [7]:
full_model_data.df.head()

,index,track_id,dataset,audio_path,label,fma_genre_top,fma_genres,fma_genres_all,file_available,sampling_rate,...,zero_crossing_rate_mean,rms_mean,chroma_stft_mean,mfccs_mean,onset,tempo,contrast,tonnetz,mfccs_min,mfccs_max
0,0,000002,fma,/project_data_source/free_music_archive/fma_sm...,hiphop,Hip-Hop,[21],[21],1.0,44100,...,0.098364,0.141225,0.424169,-1.742610,1.403928,166.708664,21.300203,0.022087,-505.499176,196.868652
1,2,000005,fma,/project_data_source/free_music_archive/fma_sm...,hiphop,Hip-Hop,[21],[21],1.0,44100,...,0.059334,0.145243,0.475249,1.318397,1.384270,99.384018,21.096722,0.005648,-528.116272,209.443054
2,3,000010,fma,/project_data_source/free_music_archive/fma_sm...,pop,Pop,[10],[10],1.0,44100,...,0.080989,0.186656,0.430406,2.244156,1.123551,112.347145,20.852177,-0.033316,-537.197205,229.155151
3,15,000140,fma,/project_data_source/free_music_archive/fma_sm...,folk,Folk,[17],[17],1.0,44100,...,0.027256,0.068459,0.450882,-3.624865,1.135973,107.666016,21.727451,0.012266,-592.895813,199.950806
4,16,000141,fma,/project_data_source/free_music_archive/fma_sm...,folk,Folk,[17],[17],1.0,44100,...,0.034120,0.100852,0.341356,-4.471582,0.881413,120.185318,23.654802,-0.003315,-536.546936,222.208649


### Extraction while saving audio and features on small sample
in this run, we saved the output of librosa load in the parquet as well.  This takes a lot of memory so only did a small example.   This is also a demo of using AudioFeatureExtractor directly for a single extraction on a subset of the data.  

In [4]:
#instantiate data 

gtza_test_data = df_filtered[df_filtered.dataset == 'gtzan']

In [5]:
#instantiate extractor
#commented in alternate return classe in get_audio_data method to include full audio
gtza_extract = AudioFeatureExtractor(gtza_test_data.head(5))

In [6]:
#run extraction
#print statement uncommented 
gtza_extract.add_audio_data_to_df()

Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00000.wav
Run Librosa Load
extracting_features
Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00001.wav
Run Librosa Load
extracting_features
Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00002.wav
Run Librosa Load
extracting_features
Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00003.wav
Run Librosa Load
extracting_features
Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00004.wav
Run Librosa Load
extracting_features
putting features to their own columns


In [7]:
#save the results
gtza_extract.save_results() 

In [10]:
#Load the model data for this version
model_data = ModelDataLoader('000')

In [11]:
#Note full librosa_load output array was also saved in this parquet
model_data.df

,dataset,audio_path,label,fma_genre_top,fma_genres,fma_genres_all,librosa_load,sampling_rate,features,spectral_centroids_mean,spectral_centroids_delta_mean,spectral_centroids_accelerate_mean,spectral_bandwidth_mean,spectral_rolloff_mean,zero_crossing_rate_mean,rms_mean,chroma_stft_mean,mfccs_mean
track_id,,,,,,,,,,,,,,,,,,
blues.00000.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[0.0073242188, 0.016601562, 0.0076293945, -0.0...",22050,"[1784.1227, -0.49052292, -0.061881434, 2002.41...",1784.122681,-0.490523,-0.061881,2002.412354,3805.723145,0.083045,0.130184,0.350129,3.121230
blues.00001.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[0.0034179688, 0.0043029785, 0.001373291, 0.00...",22050,"[1530.2617, 0.6459638, -0.0855879, 2038.9875, ...",1530.261719,0.645964,-0.085588,2038.987549,3550.713623,0.056040,0.095908,0.340849,-0.007777
blues.00002.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[0.019012451, 0.047698975, 0.029418945, -0.018...",22050,"[1552.8325, 0.3400274, 0.025039379, 1747.754, ...",1552.832520,0.340027,0.025039,1747.754028,3042.410156,0.076291,0.175473,0.363538,3.364070
blues.00003.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[-0.013000488, -0.03060913, -0.036071777, -0.0...",22050,"[1070.1534, 0.25125474, 0.07936938, 1596.4226,...",1070.153442,0.251255,0.079369,1596.422607,2184.879150,0.033309,0.141040,0.404854,0.866870
blues.00004.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[-0.0063171387, -0.009277344, -0.008331299, -0...",22050,"[1835.1285, -0.17207043, -0.24526799, 1748.410...",1835.128540,-0.172070,-0.245268,1748.410767,3579.957520,0.101461,0.091501,0.308526,-9.653034
